<a href="https://colab.research.google.com/github/IgorOlenchuk/hakaton_2022_kk/blob/main/%D0%A6%D0%9F_%D1%85%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Устанавливаем бибилиотеку Darts (https://unit8co.github.io/darts/README.html)
pip install darts

SyntaxError: ignored

In [ ]:
# Импортируем необходимые бибилиотеки
import pandas as pd

from darts import TimeSeries
from darts.models import ExponentialSmoothing, NaiveSeasonal

# Утилиты
from pathlib import Path
from datetime import datetime
import time
from tqdm import tqdm

ModuleNotFoundError: ignored

In [ ]:
# Подключаем облачное хранилище
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Фукнция, которая проходит по файлам внутри указанной директории
def do_work(directory):
    pathlist = Path(directory).glob('*.xlsx')

    for file in tqdm(pathlist):
      print('Обрабатываем файл: {file}'.format(file=file))
      file_time(file)

# Функция создания, обучения и предсказания пропущенных данных
def forcast(data, date, x, n):
    series = TimeSeries.from_dataframe(data, date, x)
    n = int(n)
    train, test = series[:-n], series[-n:]
    model = NaiveSeasonal(K=12)
    model.fit(train)
    forecast = model.predict(len(test))
    print('В колонку {col} добавляем {n} значений'.format(col=x, n=n))
    print(' '.join(forecast.pd_series().astype('str')))
    return pd.concat([train.pd_series(), forecast.pd_series()], axis=0).values

# Функция считывает указанный лист из файла и проходит по всем колонкам листа, передавай их на дальнейшую обработку
def read_data(file, sheet):
    format_datetime = lambda x: datetime.strptime(x, '%ym%m')

    data = pd.read_excel(file, 
                        sheet_name=sheet, 
                        index_col=False, 
                        engine='openpyxl', 
                        parse_dates=[0], 
                        date_parser=format_datetime)

    count_var = 0
    count_forecast = 0
    for col in data.columns:
        count_forecast = data[data[col]=='Forecast'][col].count()
        count_var = data[col].count()
        print('В колонке {col} пропущено {count_forecast} значений из {count_var}'.format(col=col, 
                                                                                          count_forecast=count_forecast, 
                                                                                          count_var=count_var))
        if count_forecast>0:
          data[col] = data[col].replace(['Forecast'], 0)
          data[col] = forcast(data, data.columns[0], col, count_forecast)
    return data

# Функция открывает указанный файл и проходит по всем листам. 
# После обработки данных, функция записывает результат в новый файл      
def file_time(file):    
    xls_file = pd.ExcelFile(file)
    out_file = str(file).replace('input', 'output')
    
    for sheet in xls_file.sheet_names:
        new_data = read_data(file, sheet)
        new_data['Unnamed: 0'] = new_data['Unnamed: 0'].dt.strftime('%ym%m')
        new_data.rename(columns = {'Unnamed: 0':''}, inplace = True)

        if sheet == 'Monthly':
            print('Добавляем лист Месяцы')            
            writer = pd.ExcelWriter(out_file)
            print('Сохраняем файл {f}'.format(f=out_file))
            new_data.to_excel(writer, sheet_name=sheet, index=False)
            writer.save()
        elif sheet == 'Quarterly':
            print('Добавляем лист Кварталы')
            my_file = Path(out_file)
            if my_file.is_file():
              writer = pd.ExcelWriter(out_file, engine='openpyxl', mode='a')
            else:
              writer = pd.ExcelWriter(out_file)
            
            print('Сохраняем файл {f}'.format(f=out_file))
            new_data.to_excel(writer, sheet_name=sheet, index=False)
            writer.save()

In [ ]:
# Путь к исходным файлам
directory = '/content/drive/MyDrive/Colab Notebooks/data/input'

do_work(directory)

0it [00:00, ?it/s]

Обрабатываем файл: /content/drive/MyDrive/Colab Notebooks/data/input/Test_example1.xlsx
В колонке Unnamed: 0 пропущено 0 значений из 120
В колонке Var1 пропущено 12 значений из 120
В колонку Var1 добавляем 12 значений
0.8219774847310676 0.8299456543802444 0.8359277260577916 0.8399197473273292 0.8449072888383683 0.8469052915010413 0.8469052915010413 0.8449032888303682 0.8449032888303682 0.8498908303414073 0.8538828516109448 0.8578748728804824
В колонке Var2 пропущено 12 значений из 120
В колонку Var2 добавляем 12 значений
2.713 2.815 3.081 3.218 3.734 3.71 3.848 3.937 4.434 4.93 5.054 5.407
В колонке Var3 пропущено 12 значений из 120
В колонку Var3 добавляем 12 значений
4.541484634301958 4.618481200047763 4.711818024863624 4.782831595588305 4.713646716824173 4.715941830500369 4.747480121705439 4.6958005274524295 4.713984333261473 4.685706727749457 4.706409550903165 4.6780116421632245
В колонке Var4 пропущено 12 значений из 120
В колонку Var4 добавляем 12 значений
0.6940227355572807 0.68

1it [00:00,  2.84it/s]

В колонке Unnamed: 0 пропущено 0 значений из 40
В колонке Var1 пропущено 4 значений из 40
В колонку Var1 добавляем 4 значений
0.263462328216347 0.32121660325488366 0.377615099667935 0.43990295177759187
Добавляем лист Кварталы
Сохраняем файл /content/drive/MyDrive/Colab Notebooks/data/output/Test_example1.xlsx
Обрабатываем файл: /content/drive/MyDrive/Colab Notebooks/data/input/Test_example2.xlsx
В колонке Unnamed: 0 пропущено 0 значений из 120
В колонке Var1 пропущено 12 значений из 120
В колонку Var1 добавляем 12 значений
0.8219774847310676 0.8299456543802444 0.8359277260577916 0.8399197473273292 0.8449072888383683 0.8469052915010413 0.8469052915010413 0.8449032888303682 0.8449032888303682 0.8498908303414073 0.8538828516109448 0.8578748728804824
В колонке Var2 пропущено 12 значений из 120
В колонку Var2 добавляем 12 значений
2.713 2.815 3.081 3.218 3.734 3.71 3.848 3.937 4.434 4.93 5.054 5.407
В колонке Var3 пропущено 0 значений из 120
В колонке Var4 пропущено 12 значений из 120
В кол

2it [00:00,  2.53it/s]

...

Обрабатываем файл: /content/drive/MyDrive/Colab Notebooks/data/input/Test_example12.xlsx
В колонке Unnamed: 0 пропущено 0 значений из 120
В колонке Var1 пропущено 0 значений из 120
В колонке Var2 пропущено 0 значений из 120
В колонке Var3 пропущено 12 значений из 120
В колонку Var3 добавляем 12 значений
0.1762596898010036 0.6276602305590764 0.6805337083499966 0.8804537781896031 0.484713367335608 0.6246003452384964 0.11150079969187476 0.9281950385452944 0.9391423083983239 0.8741044552288066 0.9004958330664748 0.9006234906497808
В колонке Var4 пропущено 13 значений из 120
В колонку Var4 добавляем 13 значений
0.7340200146438848 0.44618089138389727 0.982455590942067 0.3573482441091663 0.08828595543357043 0.016435449087477072 0.8270091264376895 0.07147179192140962 0.48772400756959877 0.8192398744301537 0.518311866114908 0.8420117231920723 0.7340200146438848
В колонке Var5 пропущено 14 значений из 120
В колонку Var5 добавляем 14 значений
0.24972024891294464 0.29721567286399897 0.053901564878

13it [00:03,  4.54it/s]

В колонке Unnamed: 0 пропущено 0 значений из 40
В колонке Var1 пропущено 5 значений из 40
В колонку Var1 добавляем 5 значений
0.14028800646468897 0.14816587208761312 0.43970264930835634 0.40479139744189363 0.3336462998664148
В колонке Var2 пропущено 4 значений из 40
В колонку Var2 добавляем 4 значений
0.2517196115176382 0.31801710240244463 0.2364174005356865 0.9939102613914286
В колонке Var3 пропущено 0 значений из 40
Добавляем лист Кварталы
Сохраняем файл /content/drive/MyDrive/Colab Notebooks/data/output/Test_example13.xlsx
Обрабатываем файл: /content/drive/MyDrive/Colab Notebooks/data/input/Test_example14.xlsx
В колонке Unnamed: 0 пропущено 0 значений из 120
В колонке Var1 пропущено 0 значений из 120
В колонке Var2 пропущено 13 значений из 120
В колонку Var2 добавляем 13 значений
0.6506730715664147 0.5579111334043947 0.6773410555273494 0.30114949785926637 0.06264071879010324 0.08539404858695554 0.5177884836366385 0.919756641654311 0.011108554002323467 0.8816076053993547 0.4705737235

14it [00:03,  3.59it/s]

В колонке Unnamed: 0 пропущено 0 значений из 40
В колонке Var1 пропущено 4 значений из 40
В колонку Var1 добавляем 4 значений
0.14816587208761312 0.43970264930835634 0.40479139744189363 0.3336462998664148
В колонке Var2 пропущено 4 значений из 40
В колонку Var2 добавляем 4 значений
0.2517196115176382 0.31801710240244463 0.2364174005356865 0.9939102613914286
В колонке Var3 пропущено 4 значений из 40
В колонку Var3 добавляем 4 значений
0.5530994826005146 0.4303115417298944 0.8609662303525952 0.43806152533937326
Добавляем лист Кварталы
Сохраняем файл /content/drive/MyDrive/Colab Notebooks/data/output/Test_example14.xlsx
